In [1]:
import pandas as pd

In [11]:
# Load the data

messages_df = pd.read_csv('../../data/exercise_2/ForumMessages.csv')
topics_df = pd.read_csv('../../data/exercise_2/ForumTopics.csv')
datasets_df = pd.read_csv('../../data/exercise_2/Datasets.csv')

/tmp/ipykernel_84182/1031707692.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  messages_df = pd.read_csv('../../data/exercise_2/ForumMessages.csv')


In [7]:
messages_df.head(1)

,Id,ForumTopicId,PostUserId,PostDate,ReplyToForumMessageId,Message,RawMarkdown,Medal,MedalAwardDate
0,97,28,62,06/22/2010 13:10:43,NaN,tes,NaN,NaN,NaN


In [12]:
# Count the lines where the message contains the word "synthetic"
synthetic_count = messages_df['Message'].str.contains('synthetic', case=False, na=False).sum()
print(f'Number of messages containing the word "synthetic": {synthetic_count}')

Number of messages containing the word "synthetic": 5243


In [20]:
# Rename columns to avoid confusion during merges
topics_df = topics_df.rename(columns={'Id': 'TopicId', 'Title': 'TopicTitle'})
datasets_df = datasets_df.rename(columns={'Id': 'DatasetId', 'Title': 'DatasetTitle', 'Slug': 'DatasetSlug'})

# 2. Step 1: Filter Topics to ONLY those belonging to Datasets
# We do an 'inner' join. If a ForumTopic doesn't match a Dataset's ForumId, it's dropped.
# This removes general forum posts and kernel comments automatically.
dataset_topics = pd.merge(topics_df, datasets_df, on='ForumId', how='inner')

# 3. Step 2: Merge the Messages into these Dataset Topics
# Now we grab the actual text comments for these validated topics.
dataset_comments = pd.merge(messages_df, dataset_topics, left_on='ForumTopicId', right_on='TopicId', how='inner')
# 4. Step 3: Search for your "Synthetic" keywords
keywords = ['synthetic', 'fake data']
pattern = '|'.join(keywords)

In [24]:
# specific_hits now contains only comments ON datasets containing your terms
specific_hits = dataset_comments[
    dataset_comments['Message'].str.contains(pattern, case=False, na=False)
]

specific_hits.iloc[200].RawMarkdown

"As mentioned, it's synthetic dataset and is expected for education purposes. \nIf it's treated as training data for building business decisions, you need to account for the possible risk."

In [23]:
print('Number of messages containing keywords:', specific_hits.shape[0])

Number of messages containing keywords: 449


In [26]:
datasets_df.columns

Index(['DatasetId', 'CreatorUserId', 'OwnerUserId', 'OwnerOrganizationId',
       'CurrentDatasetVersionId', 'CurrentDatasourceVersionId', 'ForumId',
       'Type', 'CreationDate', 'LastActivityDate', 'TotalViews',
       'TotalDownloads', 'TotalVotes', 'TotalKernels', 'Medal',
       'MedalAwardDate'],
      dtype='object')

In [34]:
import pandas as pd

# Define your keyword lists
POSITIVE_TERMS = ['synthetic'] 
NEGATIVE_TERMS = ['synthetic', 'generated', 'simulated'] 

def get_suspected_synthetic_datasets():
    print("Loading message data...")
    # 1. Load Messages and filter immediately
    msgs = pd.read_csv('../../data/exercise_2/ForumMessages.csv', usecols=['Id', 'ForumTopicId', 'Message'])
    
    pos_pattern = '|'.join(POSITIVE_TERMS)
    suspect_msgs = msgs[msgs['Message'].str.contains(pos_pattern, case=False, na=False)]
    
    print(f"Found {len(suspect_msgs)} comments mentioning 'synthetic'. Linking to datasets...")

    # 2. Load Topics and Datasets
    topics = pd.read_csv('../../data/exercise_2/ForumTopics.csv', usecols=['Id', 'ForumId', 'Title'])
    datasets = pd.read_csv('../../data/exercise_2/Datasets.csv', usecols=['Id', 'ForumId'])
    
    # --- FIX: Explicitly rename 'Id' to avoid confusion ---
    datasets = datasets.rename(columns={'Id': 'DatasetId'})

    # 3. Merge: Messages -> Topics -> Datasets
    # Left on ForumTopicId, Right on Id (Topic Id)
    suspects = suspect_msgs.merge(topics, left_on='ForumTopicId', right_on='Id', suffixes=('_msg', '_topic'))
    
    # Merge with Datasets on ForumId
    suspects = suspects.merge(datasets, on='ForumId')
    
    # Now we can safely use 'DatasetId'
    candidate_ids = suspects['DatasetId'].unique()
    print(f"Initial Candidates (Comment Flagged): {len(candidate_ids)} datasets")

    # 4. The Negative Filter (Check Descriptions AND Titles in DatasetVersions)
    print("Checking versions for forbidden terms...")
    
    versions = pd.read_csv('../../data/exercise_2/DatasetVersions.csv', 
                           usecols=['DatasetId', 'Description', 'Title', 'Subtitle'])
    
    # Filter versions to only include our candidates
    candidate_versions = versions[versions['DatasetId'].isin(candidate_ids)].copy()
    
    # Create negative search pattern
    neg_pattern = '|'.join(NEGATIVE_TERMS)
    
    # Check Description
    confessed_desc = candidate_versions[
        candidate_versions['Description'].str.contains(neg_pattern, case=False, na=False)
    ]['DatasetId'].unique()

    # Check Title
    confessed_title = candidate_versions[
        candidate_versions['Title'].str.contains(neg_pattern, case=False, na=False)
    ]['DatasetId'].unique()
    
    # Check Subtitle
    confessed_subtitle = candidate_versions[
        candidate_versions['Subtitle'].str.contains(neg_pattern, case=False, na=False)
    ]['DatasetId'].unique()
    
    # Combine all "confessions"
    ids_to_remove = set(confessed_desc).union(set(confessed_title)).union(set(confessed_subtitle))
    
    # 5. Final Subtraction
    final_ids = set(candidate_ids) - ids_to_remove
    
    # Retrieve Titles for the final result list
    final_metadata = candidate_versions[candidate_versions['DatasetId'].isin(final_ids)][['DatasetId', 'Title']].drop_duplicates()
    
    return final_metadata

# Execute
hidden_gems = get_suspected_synthetic_datasets()
print(f"Found {len(hidden_gems)} datasets that might be 'Hidden Synthetic'.")
print(hidden_gems.head())

Loading message data...
Found 5243 comments mentioning 'synthetic'. Linking to datasets...
Initial Candidates (Comment Flagged): 315 datasets
Checking versions for forbidden terms...
Found 190 datasets that might be 'Hidden Synthetic'.
      DatasetId                                     Title
239       10664                   Retail Transaction Data
961        3258                       Sign Language MNIST
1188       4880                               Four Shapes
2078      12414  Data Science for Good: Kiva Crowdfunding
2923       2964          xinjiang(Predictive Maintenance)


In [35]:
hidden_gems.columns

Index(['DatasetId', 'Title'], dtype='object')

In [ ]:
import pandas as pd

# 1. Load Datasets.csv (Only for OwnerUserId now)
# REMOVED: 'Slug' from this list
df_ds = pd.read_csv('../../data/exercise_2/Datasets.csv', 
                    usecols=['Id', 'OwnerUserId'])

# 2. Load DatasetVersions.csv (Get the Slug here)
# ADDED: 'Slug' to this list
df_versions = pd.read_csv('../../data/exercise_2/DatasetVersions.csv',
                          usecols=['DatasetId', 'Slug'])

# 3. Load Users.csv (For UserName)
df_users = pd.read_csv('../../data/exercise_2/Users.csv', 
                       usecols=['Id', 'UserName'])

# 4. Standardize Column Names
df_ds = df_ds.rename(columns={'Id': 'DatasetId', 'OwnerUserId': 'UserId'})
df_users = df_users.rename(columns={'Id': 'UserId'})

# 5. Deduplicate Versions
# DatasetVersions has many rows per dataset. We only need one Slug per DatasetId.
df_versions = df_versions.drop_duplicates(subset='DatasetId')[['DatasetId', 'Slug']]

# 6. Chain Merge
# Start with your existing results
enriched_gems = hidden_gems.merge(df_ds, on='DatasetId', how='left')
enriched_gems = enriched_gems.merge(df_versions, on='DatasetId', how='left')
enriched_gems = enriched_gems.merge(df_users, on='UserId', how='left')

# 7. Construct URL and Print
enriched_gems['UserName'] = enriched_gems['UserName'].fillna('unknown')
enriched_gems['URL'] = "https://www.kaggle.com/datasets/" + enriched_gems['UserName'] + "/" + enriched_gems['Slug']

print(f"--- Found {len(enriched_gems)} Potential Synthetic Datasets ---\n")

for index, row in enriched_gems.iterrows():
    # Use .get() to avoid errors if Title is missing from the view
    title = row.get('Title', 'Untitled') 
    print(f"Title: {title}")
    print(f"Link:  {row['URL']}")
    print("-" * 30)

In [1]:
import pandas as pd

# CONFIG
CHUNK_SIZE = 50000  # Process 50k rows at a time to save RAM
POS_TERMS = ['synthetic']
NEG_TERMS = ['synthetic', 'generated', 'simulated']
PATH_PREFIX = '../../data/exercise_2/' # Adjust to your path

def memory_safe_search():
    
    # ---------------------------------------------------------
    # STEP 1: Scan Messages for "Synthetic" (Chunked)
    # ---------------------------------------------------------
    print("1. Scanning Messages in chunks...")
    suspect_topic_ids = set()
    
    # We only keep the IDs, not the text, to save memory
    pos_pattern = '|'.join(POS_TERMS)
    
    for chunk in pd.read_csv(PATH_PREFIX + 'ForumMessages.csv', 
                             usecols=['ForumTopicId', 'Message'], 
                             chunksize=CHUNK_SIZE):
        
        # Filter this chunk
        hits = chunk[chunk['Message'].str.contains(pos_pattern, case=False, na=False)]
        
        # Add found Topic IDs to our set
        suspect_topic_ids.update(hits['ForumTopicId'].unique())

    print(f"   -> Found {len(suspect_topic_ids)} topics discussing synthetic data.")
    
    if not suspect_topic_ids: return pd.DataFrame() # Stop if nothing found

    # ---------------------------------------------------------
    # STEP 2: Map Topics -> Forums (Chunked)
    # ---------------------------------------------------------
    print("2. Mapping Topics to Forums...")
    suspect_forum_ids = set()
    
    for chunk in pd.read_csv(PATH_PREFIX + 'ForumTopics.csv', 
                             usecols=['Id', 'ForumId'], 
                             chunksize=CHUNK_SIZE):
        
        # Keep rows where Topic Id matches our suspects
        relevant = chunk[chunk['Id'].isin(suspect_topic_ids)]
        suspect_forum_ids.update(relevant['ForumId'].unique())

    print(f"   -> Mapped to {len(suspect_forum_ids)} specific forums.")

    # ---------------------------------------------------------
    # STEP 3: Map Forums -> Datasets & Users (Chunked)
    # ---------------------------------------------------------
    print("3. finding associated Datasets...")
    # We will build a small dataframe here because we need to keep the mapping
    dataset_candidates = []

    for chunk in pd.read_csv(PATH_PREFIX + 'Datasets.csv', 
                             usecols=['Id', 'ForumId', 'OwnerUserId'], 
                             chunksize=CHUNK_SIZE):
        
        # Rename Id to DatasetId immediately
        chunk = chunk.rename(columns={'Id': 'DatasetId'})
        
        # Keep rows where ForumId is in our suspect list
        hits = chunk[chunk['ForumId'].isin(suspect_forum_ids)]
        
        if not hits.empty:
            dataset_candidates.append(hits)
            
    # Combine the small pieces into one dataframe
    if not dataset_candidates: return pd.DataFrame()
    df_candidates = pd.concat(dataset_candidates)
    
    # Get the list of IDs for the next step
    candidate_ids = df_candidates['DatasetId'].unique()
    print(f"   -> Identified {len(candidate_ids)} candidate datasets.")

    # ---------------------------------------------------------
    # STEP 4: The Negative Filter & Slug Retrieval (Chunked)
    # ---------------------------------------------------------
    print("4. Checking Descriptions and getting Slugs...")
    
    neg_pattern = '|'.join(NEG_TERMS)
    valid_datasets = []
    
    for chunk in pd.read_csv(PATH_PREFIX + 'DatasetVersions.csv', 
                             usecols=['DatasetId', 'Slug', 'Title', 'Subtitle', 'Description'], 
                             chunksize=CHUNK_SIZE):
        
        # 1. Filter: Only look at rows belonging to our candidates
        chunk = chunk[chunk['DatasetId'].isin(candidate_ids)]
        
        if chunk.empty: continue

        # 2. Negative Filter: Drop rows where Forbidden Terms appear
        mask_desc = chunk['Description'].str.contains(neg_pattern, case=False, na=False)
        mask_title = chunk['Title'].str.contains(neg_pattern, case=False, na=False)
        mask_sub = chunk['Subtitle'].str.contains(neg_pattern, case=False, na=False)
        
        # Keep rows where NONE of these are true
        clean_chunk = chunk[~mask_desc & ~mask_title & ~mask_sub]
        
        if not clean_chunk.empty:
            # We only need Metadata now
            valid_datasets.append(clean_chunk[['DatasetId', 'Slug', 'Title']])

    if not valid_datasets: return pd.DataFrame()
    df_valid = pd.concat(valid_datasets).drop_duplicates(subset='DatasetId')
    
    # ---------------------------------------------------------
    # STEP 5: Attach Usernames for URL (Chunked)
    # ---------------------------------------------------------
    print("5. Resolving Usernames...")
    
    # Merge candidates with valid filtered datasets to get OwnerUserId back
    final_set = df_valid.merge(df_candidates[['DatasetId', 'OwnerUserId']], on='DatasetId', how='left')
    
    required_user_ids = final_set['OwnerUserId'].unique()
    found_users = []
    
    for chunk in pd.read_csv(PATH_PREFIX + 'Users.csv', 
                             usecols=['Id', 'UserName'], 
                             chunksize=CHUNK_SIZE):
        
        hits = chunk[chunk['Id'].isin(required_user_ids)]
        if not hits.empty:
            found_users.append(hits)
            
    df_users = pd.concat(found_users).rename(columns={'Id': 'OwnerUserId'})
    
    # Final Merge
    result = final_set.merge(df_users, on='OwnerUserId', how='left')
    
    # Build URL
    result['UserName'] = result['UserName'].fillna('unknown')
    result['URL'] = "https://www.kaggle.com/datasets/" + result['UserName'] + "/" + result['Slug']
    
    return result

# Execute
df_results = memory_safe_search()

print(f"\n--- Found {len(df_results)} Hidden Synthetic Datasets ---")
for i, row in df_results.head(10).iterrows():
    print(f"{row['Title']}\n{row['URL']}\n")

1. Scanning Messages in chunks...
   -> Found 3740 topics discussing synthetic data.
2. Mapping Topics to Forums...
   -> Mapped to 682 specific forums.
3. finding associated Datasets...
   -> Identified 315 candidate datasets.
4. Checking Descriptions and getting Slugs...
5. Resolving Usernames...

--- Found 197 Hidden Synthetic Datasets ---
Credit Card Fraud Detection
https://www.kaggle.com/datasets/unknown/creditcardfraud

Retail Transaction Data
https://www.kaggle.com/datasets/regivm/retailtransactiondata

Sign Language MNIST
https://www.kaggle.com/datasets/datamunge/sign-language-mnist

Four Shapes
https://www.kaggle.com/datasets/smeschke/four-shapes

Data Science for Good: Kiva Crowdfunding
https://www.kaggle.com/datasets/unknown/data-science-for-good-kiva-crowdfunding

xinjiang(Predictive Maintenance)
https://www.kaggle.com/datasets/yuansaijie0604/xinjiang-pm

Graduate school admission data
https://www.kaggle.com/datasets/malapatiravi/graduate-school-admission-data

Indian Liver

In [ ]:
# Tags to filter
tags = [
    'business',
    'psychology',
    'mental health',
    'health',
    'insurance',
    'healthcare',
    'retail',
    'education',
    'biology',
    'medicine',
    'sports',
    'finance',
    'economics',
    'climate',
]

In [2]:
# Write df_results to CSV
df_results.to_csv('hidden_synthetic_datasets.csv', index=False)

In [3]:
# Load the data from hidden_synthetic_datasets.csv

all_datasets = pd.read_csv('hidden_synthetic_datasets.csv')

In [7]:

import requests
import urllib.parse
import pandas as pd
import time

# List of your candidate datasets
DATASET_TITLES = all_datasets['Title'].tolist()  # Test with last 5 for brevity

def search_openalex_fulltext(dataset_name):
    base_url = "https://api.openalex.org/works"
    
    # 1. Construct the Boolean Query
    # Logic: The full text must contain "Dataset Name" AND "Kaggle"
    # We use quotes around the dataset name to ensure it's treated as a phrase
    query_string = f'"{dataset_name}" AND "Kaggle"'
    
    # 2. Build the Filter
    # fulltext.search is the specific field for body text
    params = {
        "filter": f"fulltext.search:{query_string}",
        "per-page": 5, # Keep it small for now
        "mailto": "emil.hardarson@gmail.com" # Polite pool (faster response)
    }

    print(f"Deep scanning for: [{dataset_name}]...", end=" ")
    
    try:
        r = requests.get(base_url, params=params)
        
        if r.status_code != 200:
            print(f"Error: {r.status_code}")
            return []
            
        data = r.json()
        total_hits = data['meta']['count']
        print(f"Found {total_hits} papers.")
        
        results = []
        for work in data['results']:
            # Extract useful metadata
            results.append({
                "Dataset": dataset_name,
                "Paper_Title": work['display_name'],
                "Year": work['publication_year'],
                "Cited_By_Count": work['cited_by_count'],
                "DOI": work['doi'],
                "OpenAccess_URL": work['open_access']['oa_url'] if work['open_access']['is_oa'] else None
            })
            
        return results

    except Exception as e:
        print(f"Exception: {e}")
        return []

# ---------------------------------------------------------
# EXECUTION
# ---------------------------------------------------------
all_hits = []

for title in DATASET_TITLES:
    hits = search_openalex_fulltext(title)
    all_hits.extend(hits)
    time.sleep(1) # Be polite

# Display
df = pd.DataFrame(all_hits)

if not df.empty:
    print("\n--- Confirmed Usage in Scientific Papers ---")
    # Show papers that actually used the data
    print(df[['Dataset', 'Paper_Title', 'Year']])
else:
    print("No full-text mentions found.")

Deep scanning for: [Credit Card Fraud Detection]... Found 524 papers.
Deep scanning for: [Retail Transaction Data]... Found 2 papers.
Deep scanning for: [Sign Language MNIST]... Found 20 papers.
Deep scanning for: [Four Shapes]... Found 7 papers.
Deep scanning for: [Data Science for Good: Kiva Crowdfunding]... Found 1 papers.
Deep scanning for: [xinjiang(Predictive Maintenance)]... Found 0 papers.
Deep scanning for: [Graduate school admission data]... Found 0 papers.
Deep scanning for: [Indian Liver Patient Dataset]... Found 21 papers.
Deep scanning for: [Spam Text Message Classification]... Found 1 papers.
Deep scanning for: [IBM HR Analytics Employee Attrition & Performance]... Found 23 papers.
Deep scanning for: [Black Friday]... Found 19 papers.
Deep scanning for: [HR Employee Retention]... Found 1 papers.
Deep scanning for: [Spoken Verbs]... Found 0 papers.
Deep scanning for: [Telco Customer Churn]... Found 50 papers.
Deep scanning for: [Medical Cost Personal Datasets]... Found 10

In [8]:
df.head()

,Dataset,Paper_Title,Year,Cited_By_Count,DOI,OpenAccess_URL
0,Credit Card Fraud Detection,"A Survey of Ensemble Learning: Concepts, Algor...",2022,965,https://doi.org/10.1109/access.2022.3207287,https://ieeexplore.ieee.org/ielx7/6287639/6514...
1,Credit Card Fraud Detection,Feature selection strategies: a comparative an...,2024,332,https://doi.org/10.1186/s40537-024-00905-w,https://journalofbigdata.springeropen.com/coun...
2,Credit Card Fraud Detection,A machine learning based credit card fraud det...,2022,303,https://doi.org/10.1186/s40537-022-00573-8,https://journalofbigdata.springeropen.com/trac...
3,Credit Card Fraud Detection,PREDICTIVE ANALYSIS OF HEART DISEASES WITH MAC...,2022,330,https://doi.org/10.22452/mjcs.sp2022no1.10,https://ejournal.um.edu.my/index.php/MJCS/arti...
4,Credit Card Fraud Detection,Example-dependent cost-sensitive decision trees,2015,206,https://doi.org/10.1016/j.eswa.2015.04.042,http://orbilu.uni.lu/handle/10993/25032


In [18]:
# Are there papers for the dataset titled "Remote Work & Mental Health"
df[df['Dataset'] == "Remote Work & Mental Health"]

,Dataset,Paper_Title,Year,Cited_By_Count,DOI,OpenAccess_URL


In [13]:
len(df)

241

In [24]:
# sort by Cited_By_Count descending, but less than 10 citations
df.sort_values(by='Cited_By_Count', ascending=True).head(50)

,Dataset,Paper_Title,Year,Cited_By_Count,DOI,OpenAccess_URL,Is_Accessible
6,Retail Transaction Data,A Comparative Analysis of the CRITIC and Entro...,2025,0,https://doi.org/10.14710/jmasif.16.2.73143,https://ejournal.undip.ac.id/index.php/jmasif/...,True
17,Data Science for Good: Kiva Crowdfunding,Where is waldo in statistic class? Using maps ...,2020,0,https://doi.org/10.52041/srap.20105,https://doi.org/10.52041/srap.20105,False
23,Spam Text Message Classification,Artificial Neural Networks: A Comparative Anal...,2019,0,None,http://trap.ncirl.ie/3843/1/glennconnell.pdf,True
52,HEALTHCARE PROVIDER FRAUD DETECTION ANALYSIS,Developing and Using Data Mining to Detect Hea...,2025,0,https://doi.org/10.21203/rs.3.rs-7558249/v1,https://www.researchsquare.com/article/rs-7558...,True
80,US Health Insurance Dataset,Feature Execution Graphs: A Human-AI Co-Progra...,2025,0,https://doi.org/10.1609/aaaiss.v6i1.36052,https://ojs.aaai.org/index.php/AAAI-SS/article...,False
77,Real / Fake Job Posting Prediction,Securing Online Job Platforms: A Distributed F...,2024,0,https://doi.org/10.18280/ijsse.140601,https://www.iieta.org/download/file/fid/153120,True
79,US Health Insurance Dataset,Redefining Healthcare: Internet of Things and ...,2024,0,https://doi.org/10.24940/ijird/2024/v13/i1/jan...,https://www.internationaljournalcorner.com/ind...,False
66,Logistic regression To predict heart disease,Comparative Performance Analysis of Random For...,2022,0,https://doi.org/10.1051/matecconf/202237007005,https://www.matec-conferences.org/articles/mat...,False
123,HR Analytics: Job Change of Data Scientists,Hypergraph-based data modeling for binary clas...,2022,0,https://doi.org/10.21203/rs.3.rs-1894599/v1,https://www.researchsquare.com/article/rs-1894...,True
81,Personal Loan Modeling,Optimasi Parameter Artificial Neural Network (...,2024,0,https://doi.org/10.33772/jmks.v3i3.60,https://jmks.uho.ac.id/index.php/journal/artic...,False


In [20]:
# Save to CSV
df.to_csv('datasets_with_papers.csv', index=False)

In [19]:
# How many of the open access urls are accessible for a bot?

accessible_urls = []
for url in df['OpenAccess_URL'].dropna():
    try:
        r = requests.get(url)
        if r.status_code == 200:
            accessible_urls.append(url)
    except:
        continue

len(accessible_urls)

127

In [21]:
accessible_urls

['https://journalofbigdata.springeropen.com/counter/pdf/10.1186/s40537-024-00905-w',
 'https://journalofbigdata.springeropen.com/track/pdf/10.1186/s40537-022-00573-8',
 'https://ejournal.um.edu.my/index.php/MJCS/article/download/35980/14415',
 'https://doi.org/10.29119/1641-3466.2023.172.7',
 'https://ejournal.undip.ac.id/index.php/jmasif/article/download/73143/28629',
 'http://thesai.org/Downloads/Volume10No5/Paper_75-Feature_based_Algorithmic_Analysis.pdf',
 'https://www.nature.com/articles/s42005-021-00588-2.pdf',
 'https://link.springer.com/content/pdf/10.1007/s10462-022-10192-7.pdf',
 'https://www.frontiersin.org/articles/10.3389/fpls.2022.820146/pdf',
 'https://arxiv.org/pdf/2305.19555',
 'https://link.springer.com/content/pdf/10.1007/s12553-024-00835-w.pdf',
 'http://www.thieme-connect.de/products/ejournals/pdf/10.1055/s-0042-1760247.pdf',
 'https://link.springer.com/content/pdf/10.1007/s10462-023-10652-8.pdf',
 'https://dergipark.org.tr/tr/download/article-file/2039489',
 'http

In [22]:
# Add a column to df indicating if OpenAccess_URL is accessible
df['Is_Accessible'] = df['OpenAccess_URL'].apply(lambda x: x in accessible_urls if pd.notna(x) else False)

# Save to CSV
df.to_csv('datasets_with_papers.csv', index=False)

In [25]:
# sort by Cited_By_Count descending, has OpenAccess, but less than 10 citations
df[(df['Is_Accessible']) & (df['Cited_By_Count'] < 10)].sort_values(by='Cited_By_Count', ascending=False).head(50)

,Dataset,Paper_Title,Year,Cited_By_Count,DOI,OpenAccess_URL,Is_Accessible
21,Indian Liver Patient Dataset,Makine Öğrenimi Algoritmaları Kullanarak Kalp ...,2021,9,https://doi.org/10.31590/ejosat.1012986,https://dergipark.org.tr/tr/download/article-f...,True
44,Medical Cost Personal Datasets,Performance Evaluation of Regression Models in...,2023,9,https://doi.org/10.25147/ijcsr.2017.001.1.146,https://stepacademic.net/ijcsr/article/downloa...,True
75,HR Employee Attrition,Employee Attrition: Analysis of Data Driven Mo...,2024,9,https://doi.org/10.4108/eetiot.4762,https://publications.eai.eu/index.php/IoT/arti...,True
90,Glasses or No Glasses,Comprehensive Review and Analysis on Facial Em...,2025,8,https://doi.org/10.32604/cmc.2024.058036,https://file.techscience.com/files/cmc/2025/TS...,True
50,HEALTHCARE PROVIDER FRAUD DETECTION ANALYSIS,Detection of Health Insurance Fraud using Baye...,2023,7,https://doi.org/10.18280/ijsse.130509,https://iieta.org/download/file/fid/113380,True
42,Medical Cost Personal Datasets,Medical Insurance Cost Prediction using Machin...,2022,7,https://doi.org/10.22214/ijraset.2022.47923,https://doi.org/10.22214/ijraset.2022.47923,True
105,Resume Dataset,Resume Parser and Summarizer,2023,7,https://doi.org/10.48175/ijarsct-9064,https://doi.org/10.48175/ijarsct-9064,True
51,HEALTHCARE PROVIDER FRAUD DETECTION ANALYSIS,Ensemble learning based health care claim frau...,2023,7,https://doi.org/10.11591/ijeecs.v32.i3.pp1686-...,https://ijeecs.iaescore.com/index.php/IJEECS/a...,True
144,Brain Stroke Dataset,Blockchain-enabled digital twin system for bra...,2025,7,https://doi.org/10.1186/s40708-024-00247-6,https://braininformatics.springeropen.com/coun...,True
176,Walmart Sales,Comparative Analysis of use of Machine Learnin...,2024,6,https://doi.org/10.52783/jes.1383,http://journal.esrgroups.org/jes/article/downl...,True
